```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 题目

#h(2em) 车销售商认为汽车销售量与汽油价格、贷款利率有关，两种类型汽车（普通型和豪华型）18 个月的调查资料见表 13.29，其中 $y_1$ 是普通型汽车售量（千辆），$y_2$ 是豪华型汽车售量（千辆），$x_1$ 是汽油价格（元/gal），$x_2$ 是贷款利率（%）。

#image("Chapter10_2_1.png")

#image("Chapter10_2_2.png")

#h(2em) （1）普通型和豪华型汽车分别建立如下模型：

$
  y_1 = beta_0^((1)) + beta_1^((1)) x_1 + beta_2^((1)) x_2, quad
  y_2 = beta_0^((2)) + beta_1^((2)) x_1 + beta_2^((2)) x_2,
$

给出 $beta$ 的估计值和置信区间，决定系数 $R^2, F$ 值及剩余方差等。

#h(2em) （2）用 $x_3=0, 1$ 表示汽车类型，建立统一模型：$y=beta_0 + beta_1 x_1 + beta_2 x_2 + beta_3 x_3$，给出 $beta$ 的估计值和置信区间，决定系数 $R^2, F$ 值及剩余方差等。以 $x_3=0,1$ 代入统一模型，将结果与（1）的两个模型的结果比较，解释二者的区别。

#h(2em) （3）对统一模型就每种类型汽车分别作 $x_1$ 和 $x_2$ 与残差的散点图，有什么现象，说明模型有何缺陷？

#h(2em) （4）统一模型增加二次项和交互项，考察结果有什么改进。

= 思路

#h(2em) 使用 scikit-learn 库中的线性回归模型进行建模，也可以使用 statsmodels 库中的线性回归模型进行建模。

#h(2em) 下面是代码实现：
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# 导入数据
data = np.loadtxt('Chapter10_2_1.csv', delimiter=',', skiprows=1)

idx = data[:, 0]
y1 = data[:, 1]
y2 = data[:, 2]
x1 = data[:, 3]
x2 = data[:, 4]
_X = data[:, 3:5]
N = len(idx)

# 数据可视化
plt.plot(idx, y1, '.', label='y1')
plt.legend()
plt.show()

plt.plot(idx, y2, '.', label='y2')
plt.legend()
plt.show()

plt.plot(idx, x1, '.', label='x1')
plt.legend()
plt.show()

plt.plot(idx, x2, '.', label='x2')
plt.legend()
plt.show()

# 数据预处理
X = sm.add_constant(_X)

In [ ]:
# 第一问
model1 = sm.OLS(y1, X).fit()

print(model1.summary())

In [ ]:
residuals = model1.resid

std_resid = residuals / np.std(residuals)

for index, item in enumerate(std_resid):
  if abs(item) > 2:
    print(f'异常值第{index}号:', item)
    pass
  pass

plt.plot(idx, std_resid, '.', label='std_resid')
plt.legend()
plt.show()

注意到第 13 号数据为异常值，将其删除。

In [ ]:
y1m = np.delete(y1, 13)
Xm = np.delete(X, 13, axis=0)

model1m = sm.OLS(y1m, Xm).fit()

print(model1m.summary())

residuals = model1m.resid

print('剩余方差为：', np.var(residuals))

In [ ]:
model2 = sm.OLS(y2, X).fit()

print(model2.summary())

In [ ]:
residuals = model2.resid

std_resid = residuals / np.std(residuals)

for index, item in enumerate(std_resid):
  if abs(item) > 2:
    print(f'异常值第{index}号:', item)
    pass
  pass

plt.plot(idx, std_resid, '.', label='std_resid')
plt.legend()
plt.show()

注意到第 13 号数据同样为异常值，将其删除。

In [ ]:
y2m = np.delete(y2, 13)
Xm = np.delete(X, 13, axis=0)

model2m = sm.OLS(y2m, Xm).fit()

print(model2m.summary())

residuals = model2m.resid

print('剩余方差为：', np.var(residuals))

```{=typst}

#h(2em) 综上：

$
  beta = mat(104.6455, -35.4983, -3.2563;31.3254, -8.3022, -8.3022;) \
  "置信区间" = mat(
    (63.349, 145.942),
    (-60.460, -10.537),
    (-4.185, -2.327);
    (14.595, 48.056),
    (-18.415, 1.811),
    (-1.825, -1.073);
  ) \
  y_1 = 104.6455 - 35.4983 x_1 - 3.2563 x_2 \
  y_2 = 31.3254 - 8.3022 x_1 - 8.3022 x_2 \
$

#h(2em) 对于 $y_1$，$R^2 = 0.897, F = 61.14$，剩余方差为： 13.315568691316656

#h(2em) 对于 $y_2$，$R^2 = 0.894, F = 58.96$，剩余方差为： 2.1855609333960766

```

In [ ]:
# 第二问
yn = np.hstack((y1m, y2m))
_x1 = np.delete(x1, 13)
x1n = np.hstack((_x1, _x1))
_x2 = np.delete(x2, 13)
x2n = np.hstack((_x2, _x2))
x3 = np.hstack((np.ones(N - 1), np.zeros(N - 1)))
_Xn = np.c_[x1n, x2n, x3]
Xn = sm.add_constant(_Xn)

model3 = sm.OLS(yn, Xn).fit()

print(model3.summary())

residuals = model3.resid

print('剩余方差为：', np.var(residuals))

```{=typst}
$
  beta = mat(60.8707, -21.9003, -21.9003, 14.2294;) \
  "置信区间" = mat( \
    (29.208, 92.533),
    (-41.014, -2.787),
    (-3.064, -1.641),
    (11.026, 17.433); \
  ) \
  R^2 = 0.853 \
  F = 58.14 \
  "剩余方差" = 18.45176017386221 \
  y = 60.8707 - 21.9003 x_1 - 21.9003 x_2 + 14.2294 x_3 \
$

剩余方差增大，$R^2$ 减小，$F$ 值减小，说明模型拟合效果变差。

```

In [ ]:
# 第三问
residuals = model3.resid

std_resid = residuals / np.std(residuals)

_idx = np.delete(idx, 13)

plt.plot(_idx, std_resid[:17], '.', label='std_resid_x1')
plt.legend()
plt.show()

plt.plot(_idx, std_resid[17:], '.', label='std_resid_x2')
plt.legend()
plt.show()

```{=typst}
注意到在第一问的求解中，我们已经剔除了各自模型的异常点，但在统一模型中，会增加对 $x_1$ 的异常点（第 17 号数据），说明模型无法耦合两种类型汽车的销售情况（$x_1$ 和 $x_2$），同时 $x_1$ 和 $x_2$ 各自趋势不同，说明其与 $x_3$ 存在交互作用，需要引入交互项乃至二次项。
```

In [ ]:
# 第四问
# 此处 x3**2 与 x3 等价，因而忽略
_x = [x1n * x3, x2n * x3, x1n**2, x2n**2, x1n * x2n]

_Xnn = np.c_[x1n, x2n, x3, *_x]
Xnn = sm.add_constant(_Xnn)

model4 = sm.OLS(yn, Xnn).fit()

print(model4.summary())

```{=typst}
注意到 $x_1, x_2, x_1^2$ 和 $x_1 x_2$ 的系数置信区间经过零点，将其忽略。
```

In [ ]:
_x = [x1n * x3, x2n * x3, x2n**2]

_Xnn = np.c_[x3, *_x]
Xnn = sm.add_constant(_Xnn)

Xnm = np.delete(Xnn, 17, axis=0)

ynm = np.delete(yn, 17)

model4 = sm.OLS(ynm, Xnm).fit()

print(model4.summary())

```{=typst}
最终的模型为：

$
  y = 12.4633 + 94.7619 x_3 - 39.0984 x_1 x_3 -1.9003 x_2 x_3 -0.1025 x_2^2
$

注意此处普通型对应 $x_3=1$，豪华型对应 $x_3=0$。

```